In [1]:
import dspy
import chromadb
import ujson
import random

from chromadb.utils import embedding_functions
from dspy.evaluate import SemanticF1
from dspy.retrieve.chromadb_rm import ChromadbRM
from dspy.utils import download

# Download the Data

In [2]:
download("https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_corpus.jsonl")
with open("./ragqa_arena_tech_corpus.jsonl", "r") as fp:
    corpus = [ujson.loads(line) for line in fp]

download(
    "https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_examples.jsonl"
)
with open("ragqa_arena_tech_examples.jsonl", "r") as fp:
    qa = [ujson.loads(line) for line in fp]

print(corpus[2])
len(corpus)
print(qa[2])
len(qa)

{'doc_id': 131078, 'author': None, 'text': 'http://abtevrythng.blogspot.com/2010/06/adding-cer-certificates-on-your-android.html Shows how to actually achieve this. Worked fine for me. Try it out. In this article .cer to .pfx (which is what you need on Android) conversion is given. Simple method is given using which you can convert .cer to .pfx and use it to connect to the Wi-Fi network. Plus you dont need any Key to convert .cer to .pfx!!!'}
{'question': 'why are my text messages coming up as maybe?', 'response': 'This is part of the Proactivity features new with iOS 9: It looks at info in emails to see if anyone with this number sent you an email and if it finds the phone number associated with a contact from your email, it will show you "Maybe". \n\nHowever, it has been suggested there is a bug in iOS 11.2 that can result in "Maybe" being displayed even when "Find Contacts in Other Apps" is disabled.', 'gold_doc_ids': [3956, 3957, 8034]}


2064

# Connect to ChromaDB

In [3]:
client = chromadb.PersistentClient(path="./db")
collection = client.get_or_create_collection(name="test")

# Define Retrievers

In [4]:
embed_fn = embedding_functions.DefaultEmbeddingFunction()
retriever_model = ChromadbRM(
    collection_name="test", persist_directory="./db", embedding_function=embed_fn, k=5
)

# Load LM

In [5]:
lm = dspy.LM(model="ollama_chat/llama3.1", temperature=0.0)
dspy.configure(lm=lm)

# Build RAG Module

In [6]:
class RAG(dspy.Module):
    def __init__(self):
        self.respond = dspy.ChainOfThought("context, question -> response")

    def forward(self, question):
        ret_docs = retriever_model(question)
        context = [doc["long_text"] for doc in ret_docs]
        return self.respond(context=context, question=question)

# DSPy RAG Optimization

In [7]:
qa_examples = [dspy.Example(**data).with_inputs("question") for data in qa]
random.Random(23).shuffle(qa_examples)

train, dev, test = qa_examples[:200], qa_examples[200:500], qa_examples[500:1000]
len(train), len(dev), len(test)

(200, 300, 500)

In [9]:
train[2]

Example({'question': 'how can i show typing keyboard in record screen', 'response': 'One method involves using Screenflow, a software that includes this feature and is priced at $99.  \nAdditionally, standalone apps like Keycastr and Mouseposé can be used for the same purpose.  \nAnother alternative is showing the on-screen keyboard, although this results in a different aesthetic.', 'gold_doc_ids': [6129, 6130, 6131]}) (input_keys={'question'})

In [11]:
metric = SemanticF1(decompositional=True)
tp = dspy.MIPROv2(metric=metric, auto="medium", num_threads=4)

optimized_rag = tp.compile(
    RAG(),
    trainset=train,
    max_bootstrapped_demos=2,
    max_labeled_demos=2,
    requires_permission_to_run=False,
)

2024/12/31 00:27:39 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: True
num_candidates: 19
valset size: 160

2024/12/31 00:27:39 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2024/12/31 00:27:39 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2024/12/31 00:27:39 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


 12%|█▎        | 5/40 [01:03<07:23, 12.67s/it]


Bootstrapped 2 full traces after 5 examples for up to 1 rounds, amounting to 5 attempts.
Bootstrapping set 4/19


  2%|▎         | 1/40 [00:09<06:19,  9.72s/it]


...

2024/12/31 02:25:12 INFO dspy.evaluate.evaluate: Average Metric: 15.697818740399384 / 25 (62.8%)
2024/12/31 02:25:12 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 62.79 on minibatch of size 25 with parameters ['Predictor 0: Instruction 1', 'Predictor 0: Few-Shot Set 0'].
2024/12/31 02:25:12 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [66.83, 69.62, 64.77, 66.39, 63.54, 63.32, 62.49, 69.34, 59.19, 60.45, 50.42, 65.65, 60.32, 65.22, 59.17, 60.71, 62.13, 68.81, 63.34, 68.67, 67.08, 63.75, 65.5, 62.79]
2024/12/31 02:25:12 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [62.0, 62.94, 64.95]
2024/12/31 02:25:12 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 64.95
2024/12/31 02:25:12 INFO dspy.teleprompt.mipro_optimizer_v2: =============================


2024/12/31 02:25:12 INFO dspy.teleprompt.mipro_optimizer_v2: == Minibatch Trial 25 / 25 ==



Average Metric: 16.32 / 25 (65.3%): 100%|██████████| 25/25 [02:43<00:00,  6.54s/it]

2024/12/31 02:27:56 INFO dspy.evaluate.evaluate: Average Metric: 16.317471753598515 / 25 (65.3%)
2024/12/31 02:27:56 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 65.27 on minibatch of size 25 with parameters ['Predictor 0: Instruction 11', 'Predictor 0: Few-Shot Set 6'].
2024/12/31 02:27:56 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [66.83, 69.62, 64.77, 66.39, 63.54, 63.32, 62.49, 69.34, 59.19, 60.45, 50.42, 65.65, 60.32, 65.22, 59.17, 60.71, 62.13, 68.81, 63.34, 68.67, 67.08, 63.75, 65.5, 62.79, 65.27]
2024/12/31 02:27:56 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [62.0, 62.94, 64.95]
2024/12/31 02:27:56 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 64.95
2024/12/31 02:27:56 INFO dspy.teleprompt.mipro_optimizer_v2: =============================


2024/12/31 02:27:56 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Full Eval 3 =====
2024/12/31 02:27:56 INFO dspy.teleprompt.mipro_optimizer_v2: Doing full eval on next top


Average Metric: 104.90 / 160 (65.6%): 100%|██████████| 160/160 [16:04<00:00,  6.03s/it]

2024/12/31 02:44:00 INFO dspy.evaluate.evaluate: Average Metric: 104.8971910398255 / 160 (65.6%)
2024/12/31 02:44:00 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 65.56
2024/12/31 02:44:00 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [62.0, 62.94, 64.95, 65.56]
2024/12/31 02:44:00 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 65.56
2024/12/31 02:44:00 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2024/12/31 02:44:00 INFO dspy.teleprompt.mipro_optimizer_v2: 

2024/12/31 02:44:00 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 65.56!


# Evaluate

In [12]:
evaluate = dspy.Evaluate(
    devset=dev, metric=metric, num_threads=4, display_progress=True, display_table=True
)

In [13]:
evaluate(optimized_rag)

Average Metric: 197.57 / 300 (65.9%): 100%|██████████| 300/300 [36:40<00:00,  7.34s/it]

2024/12/31 03:22:43 INFO dspy.evaluate.evaluate: Average Metric: 197.57031592560836 / 300 (65.9%)


,question,example_response,gold_doc_ids,reasoning,pred_response,SemanticF1
0,vtdecoderxpcservice taking cpu,Quit your applications one by one and monitor the process. Common ...,"[2557, 4732, 5014, 5156, 5195, 5631, 5856, 4026]",The VTDecoderXPCService is a process that might be consuming CPU r...,Try quitting your applications one by one to identify which one is...,✔️ [0.667]
1,git auto-complete for *branches* at the command line?,"You should try to update the git version to the latest, and then i...","[6715, 1766, 2902, 6154]",Based on the provided context from various users who have encounte...,"To fix the issue of not being able to autocomplete branch names, y...",✔️ [0.667]
2,what functionality do marks offer in the el capitan terminal?,"Marks in the Terminal, which began with OS X 10.11 - El Capitan, h...","[1234, 1253]",The question is asking about the functionality offered by marks in...,"Marks offer several functionalities in the El Capitan Terminal, in...",✔️ [0.857]
3,how to convert a heif/heic image to jpeg in el capitan?,Users advise that tifig is an excellent command line tool that wor...,"[7785, 680, 2705, 7201]","Based on the provided context from various sources, I will look fo...",One possible solution is to use the `sips` command-line tool that ...,✔️ [0.857]
4,why should i use a factory class instead of direct object construc...,Using factory classes along with interfaces promotes greater long-...,[4782],The question of whether to use a factory class instead of direct o...,You should use a factory class instead of direct object constructi...,✔️ [0.857]
...,...,...,...,...,...,...
295,make the menu bar never show while in full screen,Some users report that the menu bar on Mac OS X cannot be hidden o...,"[5370, 4551, 44, 6307]","Based on the provided context from various sources, I will look fo...","According to source [1], you can achieve this by unchecking the bo...",✔️ [0.400]
296,whats a good ssh tunneling client for os x?,"SSHTunnel, a free application for managing SSH tunnels, is still g...","[2186, 2809, 2814, 2817, 526, 5277, 6415, 1635, 1671, 6739, 2868, ...",Based on the provided context about SSH tunneling clients for OS X...,Core Tunnel baked by Codinn appears to be a popular and user-frien...,✔️ [0.267]
297,delete line in vi,"Use the ""dd"" command, which will remove the line, allowing you to ...","[3122, 3123, 3129, 5910, 4792, 2164]","To answer this question, I will look through the provided context ...",There are several ways to delete lines in vi: * In any POSIX-compl...,✔️ [0.667]
298,return only the portion of a line after a matching pattern,The primary tool for text manipulation and pattern replacement is ...,"[5023, 958, 6330]","Based on the provided context from various sources, I will look fo...","To return only the portion of a line after a matching pattern, you...",


65.86